In [2]:
import sys
sys.path.append("..")
from rosalind_tools.config import *
from rosalind_tools.utils import parse_fasta, Record
from itertools import permutations
from typing import List, Dict

Given: At most 50 DNA strings of approximately equal length, not exceeding 1 kbp, in FASTA format (which represent reads deriving from the same strand of a single linear chromosome).

The dataset is guaranteed to satisfy the following condition: there exists a unique way to reconstruct the entire chromosome from these reads by gluing together pairs of reads that overlap by more than half their length.

Return: A shortest superstring containing all the given strings (thus corresponding to a reconstructed chromosome).  

In [11]:
def overlap(a:str, b: str, min_length=3) -> Dict[str, int]:
    start = 0
    while True:
        start = a.find(b[:min_length], start)
        if start == -1:
            return 0 # 0 if no match
        else:
            if b.startswith(a[start:]): # exact match
                return len(a) - start # length of overlap
        start += 1
        
def scs(substrings: List[str]) -> str:
    shortest_superstring = None
    for ssperm in permutations(substrings):
        sup = ssperm[0]
        for i in range(len(ssperm) - 1):
            olen = overlap(ssperm[i], ssperm[i + 1], min_length = 3)
            sup += ssperm[i + 1][olen:]
        if shortest_superstring is None or len(sup) < len(shortest_superstring):
            shortest_superstring = sup
    return shortest_superstring

In [5]:
# Try Sample Data set
with open(data_dir/"test_assembly.txt") as f:
    records = parse_fasta(f)
    substrings = [record.seq for record in records]
    print(scs(substrings))

ATTAGACCTGCCGGAATAC


In [12]:
# Try Rosalind Dataset (Timeout)
with open(data_dir/"rosalind_long.txt") as f:
    records = parse_fasta(f)
    substrings = [record.seq for record in records]
    print(scs(substrings))

KeyboardInterrupt: 

In [7]:
# Greedy maxmial overlap
def pick_maximal_overlap(reads, k):
    reada, readb = None, None
    best_olen = 0
    for a, b in permutations(reads, 2):
        olen = overlap(a, b, min_length=k)
        if olen > best_olen:
            reada, readb = a, b
            best_olen = olen
    return reada, readb, best_olen

def greedy_scs(reads, k):
    read_a, read_b, olen = pick_maximal_overlap(reads, k)
    while olen > 0:
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        read_a, read_b, olen = pick_maximal_overlap(reads, k)
    return ''.join(reads)

In [8]:
# Try Sample Data set
with open(data_dir/"test_assembly.txt") as f:
    records = parse_fasta(f)
    substrings = [record.seq for record in records]
    print(greedy_scs(substrings, 3))

ATTAGACCTGCCGGAATAC


In [13]:
# Try Rosalind Dataset (Timeout)
with open(data_dir/"rosalind_long.txt") as f:
    records = parse_fasta(f)
    substrings = [record.seq for record in records]
    print(greedy_scs(substrings, 3))

CGTTAGATCAGCGACTTATAGCGCGGTTACGACCCACCTTTGTTAAACTAGCTAGTGAGACAAATTGCAAGTAAACCTATTGCGAGCTCTATAATTCTGGCTTTGCCCGCAGGCTCTCGAACTAGCGTTCATGAGCTGCAGAAGGCCAGTCTTAAATACAAACCGAGTTGTTTTTGCAGAGGATCTCACTAGAAGCTCGATTATACCCGCCATGGCCTTAAGGTGAAGTACAGGTTACGGCTTCATCGCTTTTATGTCGTTGGCCTGGCGGCTGTGGATGAAGAGCGTCTACCAATAATTAAAAAAATGTTTGGGAAGTATTCACTATCCTGGTCCCAAGAAATCTATACCCCGGTAGCTGGCGAACGGCGACACTTTCCGTACATGTACGTTGCGCACTTTACGAGTCTTTATGGGACCGCCAAAATATGGATACCAGAGCTAGACCACCGACCACGCCGCTTACATAATCTAAGTGCGCAACCCGAGGTTTGTCGAGCTTTGAGGAAAGAATGACCTCGTTCATATTTGGGGTTAACCCCCCCTCGCTCTCTTGACTACGAACTTATACCAGGGATACGCTTCTGCAATCAGTGCAGATAGCCCTTCATCATTCGTACCAAAGGCTGCAAAATGAGCCCGGTTCTAACGAGCGCCCGCTTAGTAAAGCGCCGACTTCACTGGAATGAAACGTCAACTGCTATAATCTGTAAAGGCCACTCTAGCCCTTTTATAATTCTATGGGCGGCGTGCAGTAATAAGGGAGGTCGCAACTGACGTGCAGTCGTTGGTCGAGGTTGCTTGTCACTCGCTCCCTGTGCCCACCGGGGTAGCGGTGTTCGGACTTATTGGAAGTACCCAGGCAAGATGAGCGTACTGGAGCCGGGGTCTCATTTTCCTAGGGCTGTAGCTTGTCGATTAAGGTCCTTTGGGTGTTCTTGGCGTGCCCCCTAGGGGGTTGGCGCAGCCTAGCAGCAAGGAGATTTCGACTTAATAGAAT